In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
cols = ['review','sentiment']
df = pd.read_csv("./data_2.csv", usecols=[1,2], names=cols)
# above line will be different depending on where you saved your data, and your file name
df.head()

,review,sentiment
0,sentiment,text
1,I just wondering what is the purpose of making...,0
2,The buzz for this film has always been about t...,0
3,I expected to enjoy a romantic comedy featurin...,0
4,Literally every aspect of this science-fiction...,0


In [2]:
df1 = df[df.sentiment == 1 ]
df2 = df[df.sentiment == 0 ]

In [ ]:
frames = [df1,df2]
df3 = pd.concat(frames)

In [ ]:
df = df3
del frames, df1, df2, df3

In [3]:
df = df.rename(columns={"review": "text"})
df

,text,sentiment
0,sentiment,text
1,I just wondering what is the purpose of making...,0
2,The buzz for this film has always been about t...,0
3,I expected to enjoy a romantic comedy featurin...,0
4,Literally every aspect of this science-fiction...,0
...,...,...
24996,This is why i so love this website ! I saw thi...,1
24997,Monstervision was a show I grew up with. From ...,1
24998,I have not read the other comments on the film...,1
24999,What a good film! Made Men is a great action m...,1


In [ ]:
df.sentiment = df.sentiment.replace("negative",0)
df.sentiment = df.sentiment.replace("positive",1)
df[df.sentiment == 0].head(10)

In [11]:
import re
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

tok = WordPunctTokenizer()

pat1 = r'@[A-Za-z0-9_]+'
pat2 = r'https?://[^ ]+'
combined_pat = r'|'.join((pat1, pat2))
www_pat = r'www.[^ ]+'
negations_dic = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}

neg_pattern = re.compile(r'\b(' + '|'.join(negations_dic.keys()) + r')\b')

def cleaning(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    try:
        bom_removed = souped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_removed = souped
    stripped = re.sub(combined_pat, '', bom_removed)
    stripped = re.sub(www_pat, '', stripped)
    lower_case = stripped.lower()
    neg_handled = neg_pattern.sub(lambda x: negations_dic[x.group()], lower_case)
    letters_only = re.sub("[^a-zA-Z]", " ", neg_handled)
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = [x for x  in tok.tokenize(letters_only) if len(x) > 1]
    return (" ".join(words)).strip()

In [12]:
df.reset_index(drop=True, inplace=True)

In [13]:
print ("Cleaning the reviews...\n")
clean_review_texts = []
for i in range(0,len(df)):
    if( (i+1)%5000 == 0 ):
        print ("Reviews %d of %d has been processed" % ( i+1, len(df) )  )                                                                 
    clean_review_texts.append(cleaning(df['text'][i]))

Cleaning the reviews...

Reviews 5000 of 25001 has been processed
Reviews 10000 of 25001 has been processed
Reviews 15000 of 25001 has been processed
Reviews 20000 of 25001 has been processed
Reviews 25000 of 25001 has been processed
CPU times: user 12.6 s, sys: 66.3 ms, total: 12.7 s
Wall time: 12.7 s


In [14]:
len(clean_review_texts)

25001

In [15]:
clean_review_texts

['sentiment',
 'just wondering what is the purpose of making movies like this the profit and to whom they are referring what intelligence must use your brain to watch something like this crap this movie is watchable by under years old children if you are adults dont try to watch it thats the reason think hollywood started to use cartoons in movies with actors like this you must forget the art of cinema be sure that you ll have tons of pop corn to consume for time to pass till this movie ends also get many colas hamburgers your laptop your cellphone this movie can be used easily in restaurant but for sure not in theater my dog who is always next to my family when watching movie left the building the sure thing is that this movie is referring to people with no demands from the cinema art the only thing that this movie can be used is for watching it when making the supermarket shopping list am giving stars for supporting the indias cinema efforts but for nothing more or less',
 'the buzz 

In [16]:
clean_df = pd.DataFrame(clean_tweet_texts,columns=['text'])
clean_df['target'] = df.sentiment
clean_df.head()

,text,target
0,sentiment,text
1,just wondering what is the purpose of making m...,0
2,the buzz for this film has always been about t...,0
3,expected to enjoy romantic comedy featuring hi...,0
4,literally every aspect of this science fiction...,0


In [18]:
clean_df.to_csv('clean_reviews_2.csv',encoding='utf-8')

In [19]:
csv = 'clean_tweet_2.csv'
df = pd.read_csv(csv,index_col=0)
df.head()

,text,target
0,sentiment,text
1,just wondering what is the purpose of making m...,0
2,the buzz for this film has always been about t...,0
3,expected to enjoy romantic comedy featuring hi...,0
4,literally every aspect of this science fiction...,0
